In [1]:
%pylab inline
import sys
sys.path.append('../')

from managers.dataManager import DataManager

Populating the interactive namespace from numpy and matplotlib


In [2]:
dataManager = DataManager(True)

In [3]:
dates = dataManager.get_future_dates().iloc[0]
dates.ANIO.astype('str') + '-' + dates.MES.astype('str').zfill(2)

'2021-04'

In [4]:
forecast = dataManager.data_forecasting_2021_lag()
forecast

2021-04


,REF,TIENDA,DATE,ANIO,MES,CANTIDAD,PRECIO,DESCUENTO(%),AREA,ALTO,...,DIFF_2,DIFF_3,DIFF_4,DIFF_5,DIFF_6,DIFF_7,DIFF_8,DIFF_9,DIFF_10,DIFF_11
188487,XA0756:00009:,PUNTO DE VENTA POBLADO,2021-04,2021,4,NaN,238718.776113,0.422540,2484.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188488,XA0756:00009:,PUNTO DE VENTA SOACHA,2021-04,2021,4,NaN,200636.000000,0.500748,2484.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188489,XA0756:00009:,VENTAS DIGITALES,2021-04,2021,4,NaN,298445.833333,0.351574,2484.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188490,XA0757:00118:,ADMINISTRACION,2021-04,2021,4,NaN,250795.000000,0.349999,2484.0,89.0,...,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
188491,XA0757:00118:,PUNTO DE VENTA LAURELES,2021-04,2021,4,NaN,250794.500000,0.500000,2484.0,89.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272254,XA0057:00140:,PUNTO DE VENTA AV 68,2022-03,2022,3,NaN,275058.944444,0.490816,2862.0,79.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272255,XA0056:00024:,PUNTO DE VENTA SOACHA,2022-03,2022,3,NaN,275059.000000,0.511199,1681.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272256,XA0056:00024:,PUNTO DE VENTA POBLADO,2022-03,2022,3,NaN,275059.000000,0.400001,1681.0,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
272257,XA0067:00118:,ADMINISTRACION,2022-03,2022,3,NaN,303832.500000,0.650000,2484.0,89.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
forecast.sort_values(['REF','ANIO','MES'])